<a href="https://colab.research.google.com/github/Shantanu88-code/Big_Data/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Tokens").getOrCreate()

In [4]:
from pyspark.ml.feature import Tokenizer

In [5]:
# Create a sample dataframe
dataframe = spark.createDataFrame([
                                   (0, "Spark is great"),
                                   (1, "I am learning Spark"),
                                   (2, "Spark is better than Hadoop") 
], ["ID", "Sentence"])

dataframe.show()

+---+--------------------+
| ID|            Sentence|
+---+--------------------+
|  0|      Spark is great|
|  1| I am learning Spark|
|  2|Spark is better t...|
+---+--------------------+



In [6]:
# Tokeanize sentence

tokenizer = Tokenizer(inputCol="Sentence", outputCol="Words")
tokenizer

Tokenizer_971084e31a92

In [7]:
tokenized_df = tokenizer.transform(dataframe)
tokenized_df.show(truncate=False)

+---+---------------------------+---------------------------------+
|ID |Sentence                   |Words                            |
+---+---------------------------+---------------------------------+
|0  |Spark is great             |[spark, is, great]               |
|1  |I am learning Spark        |[i, am, learning, spark]         |
|2  |Spark is better than Hadoop|[spark, is, better, than, hadoop]|
+---+---------------------------+---------------------------------+



In [8]:
# Create a function to return the length of a list
def word_list_length(word_list):
  return len(word_list)

In [9]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [10]:
# Create a user defined function
count_tokens = udf(word_list_length, IntegerType())

In [11]:
tokenized_df.withColumn("Tokens", count_tokens(col("Words"))).show(truncate=False)

+---+---------------------------+---------------------------------+------+
|ID |Sentence                   |Words                            |Tokens|
+---+---------------------------+---------------------------------+------+
|0  |Spark is great             |[spark, is, great]               |3     |
|1  |I am learning Spark        |[i, am, learning, spark]         |4     |
|2  |Spark is better than Hadoop|[spark, is, better, than, hadoop]|5     |
+---+---------------------------+---------------------------------+------+

